## Nhatot Preprocessing

## 1. Imports
Import necessary libraries for data manipulation and regular expressions.

In [1]:
import pandas as pd
import numpy as np
import re

## 2. Load Data
Load the raw dataset.

In [2]:
df = pd.read_csv('../Data Collection/Datasets/nhatot.com/raw/nhatot.csv')
print("Original DataFrame shape:", df.shape)
df.head()

Original DataFrame shape: (3875, 6)


,Description,Location,Price,Price per m²,Space,Title
0,"6 PN • Nhà ngõ, hẻm",Quận Thanh Xuân • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,LÊ TRỌNG TẤN_THANH XUÂN_50M_5 TẦG PHÂN LÔ_LÔ G...
1,"3 PN • Nhà ngõ, hẻm",Quận Hoàng Mai • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,BÁN NHÀ THANH LÂN – NHÀ ĐẸP 5 TẦN5 – NGÕ NÔNG ...
2,1 PN • Chung cư,Huyện Gia Lâm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,BÁN NHANH CĂN 1N +GIÁ 2.250 TR FULL NỘI THẤT T...
3,"nhiều hơn 10 PN • Nhà ngõ, hẻm",Quận Nam Từ Liêm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,CHDV MỄ TRÌ_62M2_7T TM_17P KK_DÒNG TIỀN 95TR/T...
4,"4 PN • Nhà ngõ, hẻm",Quận Nam Từ Liêm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,MỸ ĐÌNH_DÂN XÂY_39M 4T_50M Ô TÔ_2 THOÁNG_ GẦN ...


## 3. Initial Cleaning and Column Selection
Select relevant columns for the analysis and remove duplicate rows.

In [3]:
duplicates = df.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    df = df.drop_duplicates()
    print(f"Duplicates removed. New shape: {df.shape}")


Number of duplicate rows: 4
Duplicates removed. New shape: (3871, 6)


## 4. Extract Information from Description and Location
### Extract bedrooms and property type from Description


In [4]:
def extract_from_description(desc_str):
    """
    Extract number of bedrooms and property type from description string
    Expected format: "X PN • Property Type"
    """
    if pd.isna(desc_str):
        return np.nan, None
    
    desc_str = str(desc_str).strip()
    
    # Extract number of bedrooms (PN)
    bedroom_match = re.search(r'(\d+)\s*PN', desc_str)
    bedrooms = int(bedroom_match.group(1)) if bedroom_match else np.nan
    
    # Extract property type (after "•")
    property_parts = desc_str.split('•')
    property_type = property_parts[1].strip() if len(property_parts) > 1 else None
    
    return bedrooms, property_type

# Extract district and date posted from Location
def extract_from_location(location_str):
    """
    Extract district and posting date from location string
    Expected format: "District • Date"
    """
    if pd.isna(location_str):
        return None, None
    
    location_str = str(location_str).strip()
    parts = location_str.split('•')
    
    district = parts[0].strip() if len(parts) >= 1 else None
    date_posted = parts[1].strip() if len(parts) > 1 else None
    
    return district, date_posted

# Apply the extraction functions
print("\nExtracting information from Description and Location...")
df['bedrooms'], df['property_type'] = zip(*df['Description'].apply(extract_from_description))
df['district_name'], df['date_posted'] = zip(*df['Location'].apply(extract_from_location))

# Display results
print("Information extracted from Description and Location.")
print(df[['Description', 'bedrooms', 'property_type', 'Location', 'district_name', 'date_posted']].head())



Extracting information from Description and Location...
Information extracted from Description and Location.
                      Description  bedrooms property_type  \
0             6 PN • Nhà ngõ, hẻm       6.0  Nhà ngõ, hẻm   
1             3 PN • Nhà ngõ, hẻm       3.0  Nhà ngõ, hẻm   
2                 1 PN • Chung cư       1.0      Chung cư   
3  nhiều hơn 10 PN • Nhà ngõ, hẻm      10.0  Nhà ngõ, hẻm   
4             4 PN • Nhà ngõ, hẻm       4.0  Nhà ngõ, hẻm   

                          Location     district_name   date_posted  
0   Quận Thanh Xuân • 2 ngày trước   Quận Thanh Xuân  2 ngày trước  
1    Quận Hoàng Mai • 2 ngày trước    Quận Hoàng Mai  2 ngày trước  
2     Huyện Gia Lâm • 2 ngày trước     Huyện Gia Lâm  2 ngày trước  
3  Quận Nam Từ Liêm • 2 ngày trước  Quận Nam Từ Liêm  2 ngày trước  
4  Quận Nam Từ Liêm • 2 ngày trước  Quận Nam Từ Liêm  2 ngày trước  


## 5. Clean and Convert Numeric Fields

In [5]:
def extract_numeric_price(price_str):
    """
    Extract numeric price value and convert to millions VND
    Handles Vietnamese number formats (dots as thousand separators, commas as decimal separators)
    """
    if pd.isna(price_str):
        return np.nan
    
    price_str = str(price_str).strip().lower()
        
    # Extract numeric part
    numeric_match = re.search(r'([\d,.]+)', price_str)
    if not numeric_match:
        return np.nan
    
    # First remove dots (thousand separators), then replace commas with dots (decimal separators)
    numeric_value = numeric_match.group(1)
    numeric_value = numeric_value.replace('.', '')  # Remove thousand separators
    numeric_value = numeric_value.replace(',', '.')  # Convert decimal separators
    
    try:
        value = float(numeric_value)
    except ValueError:
        return np.nan
    
    # Convert based on unit
    if 'tỷ' in price_str:
        # Convert to millions
        return value * 1000.0
    elif 'triệu' in price_str or 'tr' in price_str:
        # Already in millions
        return value 
    else:
        # Assume already in millions if no unit is found
        return value

def extract_numeric_price_per_m2(price_str):
    """
    Extract numeric price per m² value and convert to millions VND/m²
    Handles Vietnamese number formats (dots as thousand separators, commas as decimal separators)
    """
    if pd.isna(price_str):
        return np.nan
    
    price_str = str(price_str).strip().lower()
    
    # Extract numeric part
    numeric_match = re.search(r'([\d,.]+)', price_str)
    if not numeric_match:
        return np.nan
    
    # First remove dots (thousand separators), then replace commas with dots (decimal separators)
    numeric_value = numeric_match.group(1)
    numeric_value = numeric_value.replace('.', '')  # Remove thousand separators
    numeric_value = numeric_value.replace(',', '.')  # Convert decimal separators
    
    try:
        value = float(numeric_value)
    except ValueError:
        return np.nan
    
    # Handle units - tr/m² is already in millions per m²
    if 'tr/m²' in price_str or 'tr/m2' in price_str or 'triệu/m²' in price_str:
        return value
    elif 'tỷ/m²' in price_str:
        # Convert billions/m² to millions/m²
        return value * 1000.0
    else:
        # Assume already in millions/m² if no unit is found
        return value

def extract_numeric_space(space_str):
    """
    Extract numeric space value in m²
    Handles Vietnamese number formats (dots as thousand separators, commas as decimal separators)
    """
    if pd.isna(space_str):
        return np.nan
    
    space_str = str(space_str).strip().lower()
    
    # Extract numeric part
    numeric_match = re.search(r'([\d,.]+)', space_str)
    if not numeric_match:
        return np.nan
    
    # First remove dots (thousand separators), then replace commas with dots (decimal separators)
    numeric_value = numeric_match.group(1)
    numeric_value = numeric_value.replace('.', '')  # Remove thousand separators
    numeric_value = numeric_value.replace(',', '.')  # Convert decimal separators
    
    try:
        value = float(numeric_value)
    except ValueError:
        return np.nan
    
    # Handle units
    if 'm²' in space_str or 'm2' in space_str:
        return value
    elif 'ha' in space_str:
        # Convert hectares to m²
        return value * 10000.0
    else:
        # Assume already in m² if no unit is found
        return value

# Apply conversions
print("\nConverting string values to numeric with proper handling of Vietnamese number formats...")
df['price_numeric'] = df['Price'].apply(extract_numeric_price)
df['price_per_m2'] = df['Price per m²'].apply(extract_numeric_price_per_m2)
df['area'] = df['Space'].apply(extract_numeric_space)

# Convert bedrooms to numeric
df['bedrooms'] = pd.to_numeric(df['bedrooms'], errors='coerce')

# Display results of the conversion
print("Numeric conversions completed.")
print("\nSample of converted values:")
comparison_df = pd.DataFrame({
    'Original Price': df['Price'].head(5),
    'Price (millions VND)': df['price_numeric'].head(5),
    'Original Price per m²': df['Price per m²'].head(5),
    'Price per m² (millions VND)': df['price_per_m2'].head(5),
    'Original Space': df['Space'].head(5),
    'Space (m²)': df['area'].head(5)
})
print(comparison_df)

# Summary statistics for the numeric fields
print("\nSummary statistics for converted numeric fields:")
print(df[['price_numeric', 'price_per_m2', 'area', 'bedrooms']].describe())



Converting string values to numeric with proper handling of Vietnamese number formats...
Numeric conversions completed.

Sample of converted values:
  Original Price  Price (millions VND) Original Price per m²  \
0        13,9 tỷ               13900.0             278 tr/m²   
1        13,9 tỷ               13900.0             278 tr/m²   
2        13,9 tỷ               13900.0             278 tr/m²   
3        13,9 tỷ               13900.0             278 tr/m²   
4        13,9 tỷ               13900.0             278 tr/m²   

   Price per m² (millions VND) Original Space  Space (m²)  
0                        278.0          50 m²        50.0  
1                        278.0          50 m²        50.0  
2                        278.0          50 m²        50.0  
3                        278.0          50 m²        50.0  
4                        278.0          50 m²        50.0  

Summary statistics for converted numeric fields:
       price_numeric  price_per_m2         area     bed

In [6]:
df.head()

,Description,Location,Price,Price per m²,Space,Title,bedrooms,property_type,district_name,date_posted,price_numeric,price_per_m2,area
0,"6 PN • Nhà ngõ, hẻm",Quận Thanh Xuân • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,LÊ TRỌNG TẤN_THANH XUÂN_50M_5 TẦG PHÂN LÔ_LÔ G...,6.0,"Nhà ngõ, hẻm",Quận Thanh Xuân,2 ngày trước,13900.0,278.0,50.0
1,"3 PN • Nhà ngõ, hẻm",Quận Hoàng Mai • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,BÁN NHÀ THANH LÂN – NHÀ ĐẸP 5 TẦN5 – NGÕ NÔNG ...,3.0,"Nhà ngõ, hẻm",Quận Hoàng Mai,2 ngày trước,13900.0,278.0,50.0
2,1 PN • Chung cư,Huyện Gia Lâm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,BÁN NHANH CĂN 1N +GIÁ 2.250 TR FULL NỘI THẤT T...,1.0,Chung cư,Huyện Gia Lâm,2 ngày trước,13900.0,278.0,50.0
3,"nhiều hơn 10 PN • Nhà ngõ, hẻm",Quận Nam Từ Liêm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,CHDV MỄ TRÌ_62M2_7T TM_17P KK_DÒNG TIỀN 95TR/T...,10.0,"Nhà ngõ, hẻm",Quận Nam Từ Liêm,2 ngày trước,13900.0,278.0,50.0
4,"4 PN • Nhà ngõ, hẻm",Quận Nam Từ Liêm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,MỸ ĐÌNH_DÂN XÂY_39M 4T_50M Ô TÔ_2 THOÁNG_ GẦN ...,4.0,"Nhà ngõ, hẻm",Quận Nam Từ Liêm,2 ngày trước,13900.0,278.0,50.0


## 6. Convert Categorical Fields to Codes
### Convert property_type and district_name to categorical codes

In [7]:
df['property_cat'] = df['property_type'].astype('category').cat.codes
df['district_cat'] = df['district_name'].astype('category').cat.codes

print("\nCategorical conversion completed.")
print("Number of unique property types:", df['property_type'].nunique())
print("Number of unique districts:", df['district_name'].nunique())


Categorical conversion completed.
Number of unique property types: 73
Number of unique districts: 28


In [14]:
df.head()

,Description,Location,Price,Price per m²,Space,Title,bedrooms,property_type,district_name,date_posted,price_numeric,price_per_m2,area,property_cat,district_cat
0,"6 PN • Nhà ngõ, hẻm",Quận Thanh Xuân • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,LÊ TRỌNG TẤN_THANH XUÂN_50M_5 TẦG PHÂN LÔ_LÔ G...,6.0,"Nhà ngõ, hẻm",Quận Thanh Xuân,2 ngày trước,13900.0,278.0,50.0,68,25
1,"3 PN • Nhà ngõ, hẻm",Quận Hoàng Mai • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,BÁN NHÀ THANH LÂN – NHÀ ĐẸP 5 TẦN5 – NGÕ NÔNG ...,3.0,"Nhà ngõ, hẻm",Quận Hoàng Mai,2 ngày trước,13900.0,278.0,50.0,68,21
2,1 PN • Chung cư,Huyện Gia Lâm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,BÁN NHANH CĂN 1N +GIÁ 2.250 TR FULL NỘI THẤT T...,1.0,Chung cư,Huyện Gia Lâm,2 ngày trước,13900.0,278.0,50.0,0,2
3,"nhiều hơn 10 PN • Nhà ngõ, hẻm",Quận Nam Từ Liêm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,CHDV MỄ TRÌ_62M2_7T TM_17P KK_DÒNG TIỀN 95TR/T...,10.0,"Nhà ngõ, hẻm",Quận Nam Từ Liêm,2 ngày trước,13900.0,278.0,50.0,68,24
4,"4 PN • Nhà ngõ, hẻm",Quận Nam Từ Liêm • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,MỸ ĐÌNH_DÂN XÂY_39M 4T_50M Ô TÔ_2 THOÁNG_ GẦN ...,4.0,"Nhà ngõ, hẻm",Quận Nam Từ Liêm,2 ngày trước,13900.0,278.0,50.0,68,24


## 7. Outlier Removal

In [9]:
def remove_outliers(df, columns, multiplier=1.5):
    """
    Remove outliers from specified columns using the IQR method
    """
    df_filtered = df.copy()
    for col in columns:
        if col in df.columns and df[col].dtype in ['int64', 'float64']:
            Q1 = df[col].quantile(0.25)
            Q3 = df[col].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - multiplier * IQR
            upper_bound = Q3 + multiplier * IQR
            df_filtered = df_filtered[(df_filtered[col] >= lower_bound) & 
                                     (df_filtered[col] <= upper_bound)]
    return df_filtered

# Remove outliers from numeric fields
print("\nRemoving outliers from numeric fields...")
numeric_cols = ['price_numeric', 'price_per_m2', 'area', 'bedrooms']
print(f"Shape before outlier removal: {df.shape}")
df_no_outliers = remove_outliers(df, numeric_cols)
print(f"Shape after outlier removal: {df_no_outliers.shape}")
print(f"Removed {len(df) - len(df_no_outliers)} rows ({((len(df) - len(df_no_outliers))/len(df))*100:.2f}% of data) as outliers.")



Removing outliers from numeric fields...
Shape before outlier removal: (3871, 15)
Shape after outlier removal: (2467, 15)
Removed 1404 rows (36.27% of data) as outliers.


## 8. Check for Missing Values and Filter

In [15]:
print("\nChecking missing values in important columns...")
print(df_no_outliers[['price_numeric', 'price_per_m2', 'area', 'bedrooms', 'property_type', 'district_name']].isnull().sum())

# Remove rows with missing area or price_per_m2 (critical for analysis)
print(f"\nShape before removing rows with missing critical values: {df_no_outliers.shape}")
df_clean = df_no_outliers[
    df_no_outliers['area'].notna() & 
    df_no_outliers['price_numeric'].notna() & 
    (df_no_outliers['area'] > 0)  # Ensure area is positive
]
print(f"Shape after removing rows with missing critical values: {df_clean.shape}")
print(f"Removed {len(df_no_outliers) - len(df_clean)} rows with missing critical values.")



Checking missing values in important columns...
price_numeric    0
price_per_m2     0
area             0
bedrooms         0
property_type    0
district_name    0
dtype: int64

Shape before removing rows with missing critical values: (2467, 15)
Shape after removing rows with missing critical values: (2467, 15)
Removed 0 rows with missing critical values.


In [18]:
df.head(1)

,Description,Location,Price,Price per m²,Space,Title,bedrooms,property_type,district_name,date_posted,price_numeric,price_per_m2,area,property_cat,district_cat
0,"6 PN • Nhà ngõ, hẻm",Quận Thanh Xuân • 2 ngày trước,"13,9 tỷ",278 tr/m²,50 m²,LÊ TRỌNG TẤN_THANH XUÂN_50M_5 TẦG PHÂN LÔ_LÔ G...,6.0,"Nhà ngõ, hẻm",Quận Thanh Xuân,2 ngày trước,13900.0,278.0,50.0,68,25


## 9. Standardize Column Names and Final Cleanup

In [20]:
df_final = df_clean.copy()
df_final.rename(columns={
    'price_numeric': 'price_converted',
    'district_name': 'district',
    'area': 'area'
}, inplace=True)

# Retain only the necessary columns for modeling
columns_to_keep = [
    'Title', 'price_converted', 'price_per_m2', 'area', 'bedrooms', 
    'property_type', 'district', 'property_cat', 'district_cat'
]
df_final = df_final[columns_to_keep]

# Display final dataset info
print("\nFinal DataFrame shape:", df_final.shape)
print("\nFinal DataFrame info:")
print(df_final.info())
print("\nFinal DataFrame head:")
print(df_final.head())

# Save the processed DataFrame
output_path = '../Data Preprocessing/nhatot_processed.csv'
df_final.to_csv(output_path, index=False)
print(f"\nProcessed DataFrame saved to '{output_path}'")

# Show summary statistics of final dataset
print("\nSummary statistics for the final dataset:")
print(df_final.describe())


Final DataFrame shape: (2467, 9)

Final DataFrame info:
<class 'pandas.core.frame.DataFrame'>
Index: 2467 entries, 1 to 3874
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Title            2467 non-null   object 
 1   price_converted  2467 non-null   float64
 2   price_per_m2     2467 non-null   float64
 3   area             2467 non-null   float64
 4   bedrooms         2467 non-null   float64
 5   property_type    2467 non-null   object 
 6   district         2467 non-null   object 
 7   property_cat     2467 non-null   int8   
 8   district_cat     2467 non-null   int8   
dtypes: float64(4), int8(2), object(3)
memory usage: 159.0+ KB
None

Final DataFrame head:
                                               Title  price_converted  \
1  BÁN NHÀ THANH LÂN – NHÀ ĐẸP 5 TẦN5 – NGÕ NÔNG ...          13900.0   
4  MỸ ĐÌNH_DÂN XÂY_39M 4T_50M Ô TÔ_2 THOÁNG_ GẦN ...          13900.0   
6  Cần bán căn góc 2PN+1,2